In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/Applications/anaconda/bin/python'

In [3]:
import findspark
findspark.init()
print findspark.find()

/usr/local/opt/apache-spark/libexec


In [4]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [5]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x8

In [6]:
sys.version

'2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]'

In [7]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [74]:
df=pd.read_csv("docsdf100-2.csv")
df.head()

,Unnamed: 0,abstract,date,doctype,headline,id,news_desk,paragraph,section,subsection,text,type
0,0,NaN,2015-11-03,article,"Know Your Risks, but Meat Still Isn�t the Enemy",56374e0a798810408f299696,Upshot,The link between processed meat and cancer is ...,The Upshot,NaN,"Know Your Risks, but Meat Still Isn�t the Enem...",News
1,1,Study in journal The Lancet concludes Chinese ...,2015-10-13,article,Study Shows Spread of Cigarettes in China,561bc3a67988101d11e0da9e,Science,Chinese men smoke one-third of all the world�s...,Health,NaN,Study Shows Spread of Cigarettes in China Chin...,News
2,2,Food and Drug Administration approves Narcan n...,2015-11-19,article,F.D.A. Approves a Nasal Spray to Combat Opioid...,564d433579881039a958bb07,National,A nasal spray designed to reverse opioid overd...,U.S.,NaN,F.D.A. Approves a Nasal Spray to Combat Opioid...,Brief
3,3,NaN,2015-11-17,article,How to Decrease Prices for an Expensive Class ...,5649c243798810095961385d,Upshot,Two professors suggest requiring biologic manu...,The Upshot,NaN,How to Decrease Prices for an Expensive Class ...,News
4,4,NaN,2015-10-07,article,Ragout With Farfalle Gets a Touch of Green,56143f92798810321b0fac91,Dining,The sweet taste and texture of broccoli leaves...,Food,NaN,Ragout With Farfalle Gets a Touch of Green The...,News


In [75]:
df.dtypes

Unnamed: 0     int64
abstract      object
date          object
doctype       object
headline      object
id            object
news_desk     object
paragraph     object
section       object
subsection    object
text          object
type          object
dtype: object

In [76]:
df['date'] = pd.to_datetime(df['date'])

In [77]:
df.dtypes

Unnamed: 0             int64
abstract              object
date          datetime64[ns]
doctype               object
headline              object
id                    object
news_desk             object
paragraph             object
section               object
subsection            object
text                  object
type                  object
dtype: object

In [78]:
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')

In [79]:
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

In [80]:
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

In [81]:
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    return nouns2, descriptives2

In [82]:
parseout = []
for index, row in df.iterrows() : 
    parseout.append(get_parts(row.text))

In [83]:
text_parts

[([[u'dna\u2019s', u'award', u'insight', u'cell']], [[u'damaged']]),
 ([[u'doctor', u'border', u'award', u'effort', u'epidemic']], [[u'outbreak']]),
 ([[u'midwife', u'inroad', u'childbirth']],
  [[u'traditional', u'southern', u'modern']]),
 ([[u'knowledge', u'health', u'news']], [[u'week\u2019s']]),
 ([[u'series', u'article', u'correspondent', u'price', u'care']],
  [[u'medical']]),
 ([[u'experience', u'soldier', u'veteran', u'deployment']], [[u'combat']]),
 ([[u'soldier', u'veteran', u'deployment']],
  [[u'female', u'veteran', u'female']]),
 ([], []),
 ([[u'oncologist']], [[u'consult']]),
 ([[u'id']], [[u'consult']]),
 ([], []),
 ([[u'concern', u'patient', u'disease']], [[u'metabolic', u'metabolic']]),
 ([], []),
 ([], []),
 ([[u'stillbirth', u'silence']], [[u'uncommon']]),
 ([[u'diagnosis', u'treatment', u'life', u'damage', u'heart']], [[u'quick']]),
 ([[u'meeting', u'doctor']], [[u'patient\u2019s']]),
 ([[u'visit']], [[u'patient\u2019s']]),
 ([], []),
 ([[u'visit']], [[u'patient\u20

In [84]:
#nouns are the first elements of the tuples
[e[0] for e in parseout[:3]]

[[[u'risk', u'enemy', u'link', u'meat', u'cancer', u'headline']],
 [[u'study',
   u'man',
   u'one-third',
   u'world\u2019s',
   u'cigarette',
   u'man',
   u'habit'],
  [u'journal',
   u'man',
   u'one-third',
   u'world',
   u'cigarette',
   u'habit',
   u'one-third',
   u'man',
   u'number',
   u'death',
   u'year']],
 [[u'nasal',
   u'spray',
   u'overdose',
   u'approval',
   u'option',
   u'drug',
   u'overdose',
   u'training'],
  [u'spray', u'overdose', u'approval', u'option', u'overdose', u'training']]]

In [85]:
ldadatardd=sc.parallelize([ele[0] for ele in parseout]).flatMap(lambda l: l)
ldadatardd.cache()
ldadatardd.take(5)

[[u'risk', u'enemy', u'link', u'meat', u'cancer', u'headline'],
 [u'study',
  u'man',
  u'one-third',
  u'world\u2019s',
  u'cigarette',
  u'man',
  u'habit'],
 [u'journal',
  u'man',
  u'one-third',
  u'world',
  u'cigarette',
  u'habit',
  u'one-third',
  u'man',
  u'number',
  u'death',
  u'year'],
 [u'nasal',
  u'spray',
  u'overdose',
  u'approval',
  u'option',
  u'drug',
  u'overdose',
  u'training'],
 [u'spray', u'overdose', u'approval', u'option', u'overdose', u'training']]

In [86]:
ldadatardd.flatMap(lambda word: word).take(5)

[u'risk', u'enemy', u'link', u'meat', u'cancer']

In [87]:
vocabtups = (ldadatardd.flatMap(lambda word: word)
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
             .map(lambda (x,y): x)
             .zipWithIndex()
).cache()

In [88]:
vocab=vocabtups.collectAsMap()
id2word=vocabtups.map(lambda (x,y): (y,x)).collectAsMap()

In [89]:
len(vocab.keys())

3814

In [90]:
#your code here
from collections import defaultdict
def get_count(s) : 
    d = defaultdict(int)
    for w in s : 
        if vocab.has_key(w) :
            i = vocab[w]
            d[i] += 1
    return d.items()
documents = ldadatardd.map(get_count)

In [91]:
documents.take(5)

[[(2757, 1), (2408, 1), (277, 1), (470, 1), (1529, 1), (31, 1)],
 [(865, 1), (99, 1), (518, 1), (2705, 1), (1301, 1), (1150, 2)],
 [(3649, 1),
  (518, 1),
  (865, 2),
  (1673, 1),
  (2252, 1),
  (2705, 1),
  (1237, 1),
  (1150, 2),
  (1695, 1)],
 [(3425, 1), (1539, 1), (2700, 2), (2317, 1), (2454, 1), (1278, 1), (2687, 1)],
 [(1278, 1), (1539, 1), (2700, 2), (2317, 1), (2454, 1)]]

In [92]:
corpus=documents.collect()

In [93]:
import gensim

In [94]:
#your code here
lda2 = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, update_every=1, chunksize=20000, passes=1)

In [95]:
lda2.print_topics()

[u'0.010*person + 0.009*health + 0.009*child + 0.007*year + 0.006*vegetable + 0.006*vitamin + 0.006*source + 0.006*drug + 0.005*cancer + 0.005*disease',
 u'0.013*health + 0.010*recipe + 0.008*food + 0.007*person + 0.007*vegetable + 0.007*study + 0.006*week + 0.006*time + 0.006*dish + 0.006*year']